In [1]:
import numpy as np
import random
from gurobipy import *

# Discrete uniform distribution
def DU(lower, upper):
    result = np.random.randint(low=lower, high=upper)
    return result

In [2]:
# AIRPORT GENERATION
class Airport:
    def __init__(self, number_of_gates, number_of_aircrafts, time_horizon, max_duration, apron_distance):
        
        self.m = number_of_gates
        self.n = number_of_aircrafts
        self.time_horizon = time_horizon
        self.max_duration = max_duration
        #self.NA = NA # minimum number of aircraft that must be assigned to the apron.
        
        # the set of domestic (fixed gates + apron)  and the set of international (fixed gates + apron)
        self.K_D , self.K_I = np.array(self.generate_gates(number_of_gates)) 
        self.K = np.union1d(self.K_D, self.K_I) 

        self.time_intervals_i = self.get_aircrafts_intervals(n, time_horizon , max_duration)
        # Distance between gate k and the entrance/exit of the airport
        self.e_dk = np.array(self.generate_e_dk(self.K, apron_distance)) 
        # transfer and non transfer passengers
        self.p_ij = np.reshape(np.array(self.transfer_passengers(n)), (n, n)) 
        self.n_ti = np.array(self.non_transfer_passengers(n)) 
        self.e_i, self.f_i = self.split_non_transfer_passengers(self.n_ti)
        
        self.I = np.array(self.generate_aircraft_type(n))
        self.T_D = self.domestic_overlapping_aircrafts(self.I,self.time_intervals_i, time_horizon)
        self.T_I = self.international_overlapping_aircrafts(self.I,self.time_intervals_i, time_horizon)

        self.d_kl = self.generate_d_kl(m)

        # SMALL-size FC formulation parameters
        #average distances between a gate and other gates
        self.d_k_aveto =  self.d_k_aveto(self.K , self.d_kl)
        self.d_k_avefrom =  self.d_k_avefrom(self.K , self.d_kl)


  

    def generate_gates(self, number_of_gates):
            K_D = []
            K_I = []
            # check that the number_of_gates is even 
            if(number_of_gates % 2 != 0):
                raise Exception("Sorry, the number of gates must be even and greater than 0")
            for i in range(0, number_of_gates // 2):
                K_D.append(i)
            K_D.append(number_of_gates)
            for j in range(number_of_gates//2 , number_of_gates):
                K_I.append(j)
            K_I.append(number_of_gates)
            return [K_D , K_I]

    # generate time intervals for our aircrafts (time from arrival to departure)
    def get_aircrafts_intervals(self, number_of_aircrafts , max_time , max_duration):
        time_intervals = []
        for i in range(0,number_of_aircrafts):
            a_i = DU(0,max_time)
            dur_i = max_duration + DU(0,max_duration)
            #dur_i = 30 + DU(0,30)
            interval = [a_i, a_i+dur_i]
            time_intervals.append(interval)
        return time_intervals

    # generate e_dk 
    def generate_e_dk(self, K, apron_distance):
        e_dk = []
        for j in range(2):
            for i in range(0 ,len(K)//2):
                e_dk.append(3 + (2 * i))
        e_dk.append(apron_distance)
        return e_dk

    # transfer passengers and non transfer_passengers
    def transfer_passengers(self, number_of_aircrafts):
        p_ij = []
        for i in range(0,number_of_aircrafts * (number_of_aircrafts)):
            if (i % (number_of_aircrafts + 1) == 0):
                p_ij.append(0)
            else:
                p_ij.append(DU(0,(200/number_of_aircrafts)+1))
        return p_ij

    def non_transfer_passengers(self, number_of_aircrafts):
        n_ti = []
        for i in range(0,number_of_aircrafts):
            n_ti.append( DU(0, 100))
        return n_ti 

    # distribution of non transfer passengers
    def split_non_transfer_passengers(self, n_ti):
        e_i = []
        f_i = []
        for i in range(0,len(n_ti)):
            number = random.randint(0,n_ti[i])
            e_i.append(number)
            f_i.append(n_ti[i]- number)
        return [e_i,f_i]
        
    # generate_aircraft_type domestic or international
    def generate_aircraft_type(self, number_of_aircrafts):
        g_of_i = []
        for i in range(0,number_of_aircrafts):
            status = DU(0,2)
            if status == 0: 
                g_of_i.append('D')
            else:
                g_of_i.append('I')
        return g_of_i

    # domestic overlapping aircrafts 
    def get_aicrafts_of_type(self, type , I):
        aircraft_idexes = []
        for i in range(0,len(I)):
            if(I[i] == type):
                aircraft_idexes.append(i)
        return aircraft_idexes

    def is_overlaping(self, time_interval , aircraft_interval):
        if ((aircraft_interval[1] < time_interval[0]) or (aircraft_interval[0] > time_interval[1] )):
            return False
        return True

    def domestic_overlapping_aircrafts(self, I,time_intervals_i, max_time):
        domestic_indexes = self.get_aicrafts_of_type('D', I)
        T_D = []
        for t in range(0,max_time):
            T_D_t = []
            for index in domestic_indexes:
                time_interval = [t , t + 1]
                aircraft_interval = time_intervals_i[index]
                status = self.is_overlaping(time_interval, aircraft_interval)
                if(status== True):
                    T_D_t.append(index)
            T_D.append(T_D_t)    
        for t in range(0,len(T_D)):
            if len(T_D[t]) < 2:
                T_D[t] = []
        return T_D

    def international_overlapping_aircrafts(self, I,time_intervals_i, max_time):
        domestic_indexes = self.get_aicrafts_of_type('I', I)
        
        T_D = []
        for t in range(0,max_time):
            T_D_t = []
            for index in domestic_indexes:
                time_interval = [t , t + 1]
                aircraft_interval = time_intervals_i[index]
                status = self.is_overlaping(time_interval, aircraft_interval)
                if(status== True):
                    T_D_t.append(index)
            T_D.append(T_D_t)    
        for t in range(0,len(T_D)):
            if len(T_D[t]) < 2:
                T_D[t] = []
        return T_D
        
    # distance traveled when going from gate 𝑘 to gate 𝑙.
    def generate_d_kl(self, numbers_of_gates):
        A = np.zeros((numbers_of_gates,numbers_of_gates))
        for i in range(len(A)//2):
            for j in range(len(A)):
                if( j < len(A)//2 ):
                    A[i,j] = abs(j-i)*2 
                if (j >= len(A)//2):
                    A[i,j] = i * 2 + 4 + (j-(len(A)//2)) * 2
        A_top_left = A[0:len(A)//2 , 0:len(A)//2]
        B = np.transpose(A)
        A_bottom_left = B[len(B)//2 : , 0: len(B)//2]
        E = np.concatenate((A_top_left,A_bottom_left ), axis=1)
        F = np.concatenate((A_bottom_left,A_top_left ), axis=1)
        H = np.concatenate((E,F ), axis=0)
        
        row = np.full((1,len(A)), 30)
        column = np.full((len(A)+1, 1), 30)
        column[-1] = 0
        H = np.append(H , row , axis = 0)
        d_kl = np.append(H , column , axis = 1)
        return d_kl

    # function for the FC formulation
    def d_k_aveto_k(self, k , gates , d_kl):
        summation = 0
        column = d_kl[: , k]
        summation = sum(column)            
        average = summation / len(gates)
        return average 
    
    def d_k_aveto(self, gates , d_kl):
        d_k_aveto = []
        for i in range(0, len(gates)):
            d_k_aveto.append(self.d_k_aveto_k(i , gates , d_kl))
        return d_k_aveto

    def d_k_avefrom(self, gates , d_kl):
        d_k_avefrom = []
        for i in range(0, len(gates)):
            d_k_avefrom.append(self.d_k_avefrom_k(i , gates , d_kl))
        return d_k_avefrom

    def d_k_avefrom_k(self, k , gates, d_kl):
        summation = 0 
        row = d_kl[k]
        summation = sum(row)
        average = summation / len(gates)
        return average

    def g(self, i):
        if(self.I[i] == 'D'):
            return  self.K_D
        else:
            return  self.K_I 
    
    def set_NA(self,NA):
        self.NA = NA

        

In [3]:
# define the functions to find NA

def generate_arcs(arr):
    arcs = []
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            arcs.append((arr[i], arr[j]))
    return arcs


def get_domestic_aircrafts(aircrafts):
    domestic_aircrafts = []
    for i in range(0, len(aircrafts)):
        if aircrafts[i] == 'D':
            domestic_aircrafts.append(i)
    return domestic_aircrafts

def get_international_aircrafts(aircrafts):
    international_aircrafts = []
    for i in range(0, len(aircrafts)):
        if aircrafts[i] == 'I':
            international_aircrafts.append(i)
    return international_aircrafts

def create_graph(aircrafts, aircraft_type , aircrafts_schedule):
    # lets create the nodes
    nodes = []
    for i in range(0, len(aircrafts)):
        if aircrafts[i] == aircraft_type:
            nodes.append(i)
    # let's create all possibile arcs
    all_arcs = generate_arcs(nodes)
    # let's find all non overlaping arcs
    unique_arrays = set(map(tuple, filter(None, aircrafts_schedule)))
    overlaping_arcs = list(unique_arrays)
    non_overlapping_arcs = [arc for arc in all_arcs if arc not in overlaping_arcs]

    return non_overlapping_arcs

def find_NA_domestic(domestic_nodes , ID , KD):
    # Set up the model
    model = Model('Maximum Cost Network Flow domestic case 2')

    # Define decision variables
    z = model.addVars(domestic_nodes, vtype=GRB.BINARY, name='z')
    # Set up the objective function
    model.setObjective(sum(z[i,j] for i,j in domestic_nodes), GRB.MAXIMIZE)

    # Set up the constraints
    A_D_0j = [(i,j) for (i,j) in domestic_nodes if i == domestic_nodes[0][0]]
    model.addConstr(sum(z[domestic_nodes[0][0],j] for j in [t[1] for t in A_D_0j]) <= len(KD)-1, 'source_flow')

    
    nodes_list = list(set([n for pair in domestic_nodes for n in pair]))
    nodes_list = sorted(nodes_list)
    last_node = nodes_list[-1]
    nodes_with_ID = [t for t in domestic_nodes if t[1] == last_node]
    model.addConstr(sum(z[i,last_node] for i in [t[0] for t in nodes_with_ID]) <= len(KD)-1, 'sink_flow')
    

    grouped_dict = {}
    for t in domestic_nodes:
        if t[0] not in grouped_dict:
            grouped_dict[t[0]] = [t]
        else:
            grouped_dict[t[0]].append(t)
    international_array = []
    for k, v in grouped_dict.items():
        international_array.append(v)

    for row in international_array:
         in_sum = sum(z[i, j] for i,j in row)
         model.addConstr(in_sum <= 1 , name ="constraint")


    model.write("test_domestic.lp")

    model.update()
    model.optimize()
    return model

def find_NA_international(international_nodes , ID , KD):
    # Set up the model
    model = Model('Maximum Cost Network Flow domestic case 2')

    # Define decision variables
    z = model.addVars(international_nodes, vtype=GRB.BINARY, name='z')
    # Set up the objective function
    model.setObjective(sum(z[i,j] for i,j in international_nodes), GRB.MAXIMIZE)

    # Set up the constraints
    A_D_0j = [(i,j) for (i,j) in international_nodes if i == international_nodes[0][0]]
    model.addConstr(sum(z[international_nodes[0][0],j] for j in [t[1] for t in A_D_0j]) <= len(KD)-1, 'source_flow')

    
    nodes_list = list(set([n for pair in international_nodes for n in pair]))
    nodes_list = sorted(nodes_list)
    last_node = nodes_list[-1]
    nodes_with_ID = [t for t in international_nodes if t[1] == last_node]
    model.addConstr(sum(z[i,last_node] for i in [t[0] for t in nodes_with_ID]) <= len(KD)-1, 'sink_flow')
    

    grouped_dict = {}
    for t in international_nodes:
        if t[0] not in grouped_dict:
            grouped_dict[t[0]] = [t]
        else:
            grouped_dict[t[0]].append(t)
    international_array = []
    for k, v in grouped_dict.items():
        international_array.append(v)

    for row in international_array:
         in_sum = sum(z[i, j] for i,j in row)
         model.addConstr(in_sum <= 1 , name ="constraint")


    model.write("test_international.lp")

    model.update()
    model.optimize()
    return model

# Build maximum network flow problem 
def find_and_set_NA(airport):
    aircrafts = airport.I

    # # generate graph for domestic nodes
    domestic_nodes = create_graph(aircrafts, 'D' , airport.T_D)
    print("domestic_nodes: ", domestic_nodes)
    # # generate graph for international nodes
    international_nodes = create_graph(aircrafts, 'I' , airport.T_I)  

    # #define numer of domestic and international aircrafts
    ID = len(get_domestic_aircrafts(aircrafts))
    II = len(get_international_aircrafts(aircrafts))
    # # define set of domestic and international gates apron included
    KD = airport.K_D
    KI = airport.K_I

    na_domestic = ID - find_NA_domestic(domestic_nodes , ID , KD).objval
    na_international = II - find_NA_international(international_nodes , II , KI).objval
    print("na_domestic" , na_domestic)
    print("na_international" , na_international)
    NA = airport.n - na_domestic - na_international
    #NA  = airport.n - airport.m
    airport.set_NA(NA)
    return NA


In [4]:
# IFS  formulation
def IFS(airport):
    IFS_model = Model(name= 'IFS')
    n = airport.n
    m = airport.m

    # decision variables
    x = IFS_model.addVars(n,m+1, vtype = GRB.BINARY, name = "x")

    # Objective function
    obj_fn = sum( ( airport.d_k_aveto[k] * (sum(airport.p_ij[j,i] for j in range(0,n) )) + 
             airport.d_k_avefrom[k] * (sum(airport.p_ij[i,j] for j in range(0,n) )) + 
             (airport.e_dk[k] * (airport.e_i[i] + airport.f_i[i])))  * x[i,k] for i in range(0,n) for k in airport.g(i) )
    
    IFS_model.setObjective(obj_fn, GRB.MINIMIZE)

    # constraints
    IFS_model.addConstrs( (sum( x[i,k] for k in airport.g(i)) == 1 for i in range(0, n)) )

    for t in range(0, airport.time_horizon):   
        IFS_model.addConstrs( (sum( x[i,k] for i in airport.T_D[t]) <= 1 for k in airport.K_D[0:-1] ) )

    for t in range(0, airport.time_horizon):   
        IFS_model.addConstrs( (sum( x[i,k] for i in airport.T_I[t]) <= 1 for k in airport.K_I[0:-1] ) )

    IFS_model.addConstrs( (sum( x[i,s] for i in range(n)) == airport.NA for s in range(m,m+1)) )
    
    IFS_model.update()
    #IFS_model.write('IFS_model.lp')
    IFS_model.optimize()
    return IFS_model

# run the IFS formulation 


In [5]:
# x_ik each row represent the aircraft to which gate k is assigned.
def get_x_ik(IFS):
    x_ik = [] # aircraft i assigned to gate k
    for v in IFS.getVars():
        x_ik.append(v.X)
    x_ik = np.array(x_ik)
    x_ik = np.reshape(x_ik, (n, m+1))
    return x_ik


In [6]:
#FC_FORMULATION
def FC(airport ,new_k_g_i ):
    FC_MODEL = Model(name= 'FC_MODEL')
    n = airport.n
    m = airport.m
    
    # Add decision variables
    w = FC_MODEL.addVars(n,m+1,m+1, vtype = GRB.CONTINUOUS, name = "w")
    x = FC_MODEL.addVars(n,m+1, vtype = GRB.BINARY, name = "x")

    # Add Objective function
    obj_fn =( sum(  airport.d_kl[k,l] * w[i , k , l ] for i in range(0, n) for k in new_k_g_i[i] for l in airport.K  ) + 
              sum( (airport.e_i[i] + airport.f_i[i])* airport.e_dk[k]*x[i,k] for i in range(0,n) for k in new_k_g_i[i] ) )
    FC_MODEL.setObjective(obj_fn, GRB.MINIMIZE)
    
    # add constraints 10 , 11, 12
    for i in range(0, n):   
        for k in new_k_g_i[i]:
            w_i_k_l = sum(w[i,k,l] for l in airport.K)
            left_side = sum(airport.p_ij[i,j] for j in range(0,n)) * x[i,k]
            expr = (left_side == w_i_k_l)
            FC_MODEL.addConstr( expr)
    
    for i in range(0, n):   
        for k in airport.K:
            w_i_l_k = sum(w[i,l,k] for l in new_k_g_i[i])
            left_side = sum(airport.p_ij[i,j] * x[j,k] for j in range(0,n)) 
            expr2= (left_side == w_i_l_k)
            FC_MODEL.addConstr( expr2)

    for i in range(0, n):   
        for k in new_k_g_i[i]:
            expr3 = (w[i,k,l] >= 0 for l in airport.K)
            FC_MODEL.addConstrs(expr3)
    
    # add constraints 3-7
    FC_MODEL.addConstrs( (sum( x[i,k] for k in new_k_g_i[i]) == 1 for i in range(0, n)) )

    for t in range(0, airport.time_horizon):   
        FC_MODEL.addConstrs( (sum( x[i,k] for i in airport.T_D[t]) <= 1 for k in airport.K_D[0:-1] ) )

    for t in range(0, airport.time_horizon):   
        FC_MODEL.addConstrs( (sum( x[i,k] for i in airport.T_I[t]) <= 1 for k in airport.K_I[0:-1] ) )

    FC_MODEL.addConstrs( (sum( x[i,s] for i in range(n)) == airport.NA for s in range(m,m+1)) )

#add constraints 13 14

    FC_MODEL.update()
    FC_MODEL.optimize()
    #FC_MODEL.write("FC_model.lp")
    return FC_MODEL


# FC functions
def is_beta_valid(airport, beta, i):
    g_i = airport.g(i)
    if beta < len(g_i):
        return True
    return False

def take_beta_gates(airport, beta, gates, gates_assigned_for_i, i):

    if is_beta_valid(airport, beta, i) == False:
        beta = len(airport.g(i)) - 1
    
    # take first beta - 1 gates
    gates = gates[0:beta-1]
    assigned_gates_indices = gates_assigned_for_i.index(1)
    if (((assigned_gates_indices in gates) != True) and (assigned_gates_indices != len(gates_assigned_for_i)-1)):
        gates[beta-1 - 1] = assigned_gates_indices
    gates.append(len(airport.K) - 1)
    return gates

def new_k_g_i(airport, beta, IFS, x_ik):
    # Convert x_ik to a list of lists of integers
    x_ik = [[round(x) for x in row] for row in x_ik]
    h = 0
    new_k_g_i  = []
    for aircraft in x_ik:
        gate_costs = {}
        for k in airport.K:
            gate_costs[k] = ( airport.d_k_aveto[k] * (sum(airport.p_ij[j,h] for j in range(0,n) )) + 
                                airport.d_k_avefrom[k] * (sum(airport.p_ij[h,j] for j in range(0,n) )) + 
                                (airport.e_dk[k] * (airport.e_i[h] + airport.f_i[h])))
        sorted_gates = dict(sorted(gate_costs.items(), key=lambda x: x[1]))
        gates = [int(key) for key in sorted_gates]
        new_gates = take_beta_gates(airport, beta, gates, x_ik[h], h)
        h = h + 1
        new_k_g_i.append(new_gates)

    return new_k_g_i

In [7]:
#define METAHEURISTIC
import time

def metaheuristic(airport , beta , IFS , x_ik):
    previous_result = IFS.objVal #upper_bound
    k_g_i = new_k_g_i(airport , beta , IFS , x_ik)
    improvement = 1
    start  = time.time()
    finish = 0
    time_diff = finish - start
    while((improvement > 0) & (time_diff < 3600)):
        # solve FC with the new_k_g_i
        FC_result_new = FC(airport , k_g_i)
        # update improvement
        improvement = previous_result - FC_result_new.objVal
        # update time
        finish = time.time()
        time_diff = finish - start
        # set beta to b + 1
        beta = beta + 1
        x_ik = []
        for v in FC_result_new.getVars():
            if "x" in v.VarName:
                x_ik.append(v.X)
        x_ik = np.array(x_ik)
        x_ik = np.reshape(x_ik, (airport.n, airport.m + 1))
        # update new_k_g_i
        k_g_i = new_k_g_i(airport , beta , FC_result_new , x_ik)
        previous_result = FC_result_new.objVal
    return FC_result_new.objVal


In [8]:
import time
# airports generation
apron_distance = 30
data = []
for m in [8,10,12]:
    for n in [15, 20, 25]:
        for i in range(0,10):
            print("i = " , i , " m = " , m , " n = " , n, "--------------------------------------------------------------")
            airport = Airport(m, n, 300, 30, apron_distance)

            # find NA
            NA = find_and_set_NA(airport)
            # process IFS formulation
            IFS_formulation = IFS(airport)
            # get x_ik
            x_ik = get_x_ik(IFS_formulation)
            # define beta (run experiment with multiple betas)
            initial_beta = 4
            beta_max = airport.m - 1
            step = 1
            # beta_list = [round(x, 1) for x in list(np.arange(initial_beta, beta_max + step, step))]
            beta_list = [4,5,6,7] # comment this if you want a dynamic beta choice, and uncomment the code above. this choice is done in order to get less experimentation time
            for beta in beta_list:
                # run heuristic FC x
                start_time  = time.time()
                walking_distance = metaheuristic(airport , beta , IFS_formulation , x_ik)
                end_time  = time.time()
                time_diff = end_time - start_time
            
                #add data to list
                data.append([m, n, NA, walking_distance, time_diff])

i =  0  m =  8  n =  15 --------------------------------------------------------------
domestic_nodes:  [(0, 2), (0, 4), (0, 5), (0, 6), (0, 13), (2, 4), (2, 5), (2, 6), (2, 7), (2, 12), (4, 5), (4, 6), (4, 7), (4, 12), (4, 13), (5, 6), (5, 7), (5, 13), (6, 7), (6, 13), (7, 12), (7, 13), (12, 13)]
Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-19
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 23 columns and 34 nonzeros
Model fingerprint: 0x61f35b57
Variable types: 0 continuous, 23 integer (23 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 9 rows and 2

In [15]:
# m, n, NA, walking_distance, and time_diff
# convert data into an array of arrays so that it can calculate the mean and standard deviation for each (m,n) points

# initialize an empty dictionary
groups = {}

# loop over the data and add each row to the corresponding group
for row in data:
    key = (row[0], row[1])
    if key in groups:
        groups[key].append(row)
    else:
        groups[key] = [row]

# convert the dictionary values into an array
data_arr = np.array(list(groups.values()))
# print(data_arr)

In [16]:
# RESULTS FROM experiment

import pandas as pd
import numpy as np

# Create an empty DataFrame with column names
df = pd.DataFrame(columns=['m', 'n', 'Mean Walking Distance', 'Std Walking Distance', 'Min Walking Distance', 'Max Walking Distance',
                           'Mean Time Diff', 'Std Time Diff', 'Min Time Diff', 'Max Time Diff'])

# Populate the DataFrame with experiment results
for experiment in data_arr:
    walking_distances = np.array([row[3] for row in experiment])
    walking_distance_mean = np.mean(walking_distances)
    walking_distance_std = np.std(walking_distances)
    walking_distance_min = np.min(walking_distances)
    walking_distance_max = np.max(walking_distances)

    time_diff = np.array([row[4] for row in experiment])
    time_diff_mean = np.mean(time_diff)
    time_diff_std = np.std(time_diff)
    time_diff_min = np.min(time_diff)
    time_diff_max = np.max(time_diff)

    # Create a new DataFrame with the experiment results
    experiment_df = pd.DataFrame({'m': [experiment[0][0]], 'n': [experiment[0][1]],
                                  'Mean Walking Distance': [walking_distance_mean], 'Std Walking Distance': [walking_distance_std],
                                  'Min Walking Distance': [walking_distance_min], 'Max Walking Distance': [walking_distance_max],
                                  'Mean Time Diff': [time_diff_mean], 'Std Time Diff': [time_diff_std],
                                  'Min Time Diff': [time_diff_min], 'Max Time Diff': [time_diff_max]})

    # Append the new DataFrame to the main DataFrame using concat
    df = pd.concat([df, experiment_df], ignore_index=True)

# Print the DataFrame containg experimen results
print(df)


      m     n  Mean Walking Distance  Std Walking Distance  \
0   8.0  15.0                24866.1           3344.414491   
1   8.0  20.0                35501.1           3185.482740   
2   8.0  25.0                43600.8           4020.614102   
3  10.0  15.0                27954.0           3256.734561   
4  10.0  20.0                34904.1           3612.941501   
5  10.0  25.0                45567.0           4833.569426   
6  12.0  15.0                24822.6           2156.021670   
7  12.0  20.0                35039.1           3708.693151   
8  12.0  25.0                42846.6           2163.289911   

   Min Walking Distance  Max Walking Distance  Mean Time Diff  Std Time Diff  \
0               20370.0               32298.0        0.623214       0.170331   
1               28713.0               39477.0        1.530620       0.544526   
2               38064.0               50307.0        3.012804       0.691691   
3               21792.0               32385.0        1.7614